This notebook demonstrates creating and loading a custom CUDA extension.

In [1]:
import torch, os, math, gzip, pickle
from pathlib import Path

from torch import tensor
from torch.utils.cpp_extension import load_inline

## CUDA setup

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'
# Get the CUDA capability of the current device
if torch.cuda.is_available():
    major, minor = torch.cuda.get_device_capability()
    # Set the environment variable with the detected architecture
    os.environ['TORCH_CUDA_ARCH_LIST'] = f"{major}.{minor}"
    print(f"Setting TORCH_CUDA_ARCH_LIST to: {os.environ['TORCH_CUDA_ARCH_LIST']}")
else:
    print("CUDA is not available. The warning about TORCH_CUDA_ARCH_LIST is not relevant in this case.")

Setting TORCH_CUDA_ARCH_LIST to: 7.5


In [3]:
%pip install -q wurlitzer ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 9.0 MB/s eta 0:00:00


In [4]:
%load_ext wurlitzer

In [5]:
def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
                       extra_cuda_cflags=["-O2"] if opt else [], verbose=verbose, name="inline_ext")

In [6]:
cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>
'''

In [7]:
cuda_src = cuda_begin + r'''
__global__ void hello_world_kernel() {
    printf("Hello World from CUDA!\n");
}

void call_hello_world_kernel() {
    hello_world_kernel<<<1, 1>>>();
    C10_CUDA_KERNEL_LAUNCH_CHECK();
}

'''

In [8]:
cpp_src = "void call_hello_world_kernel();"

In [9]:
module = load_cuda(cuda_src, cpp_src, ['call_hello_world_kernel'], verbose=True)

Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py311_cu124/inline_ext...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu124/inline_ext/build.ninja...
Building extension module inline_ext...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=inline_ext -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.11/dist-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /usr/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /root/.cache/torch_extensions/py311_cu124/inline_ext/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=inline_ext -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/include

Loading extension module inline_ext...


In [10]:
dir(module)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'call_hello_world_kernel']

In [12]:
module.call_hello_world_kernel()

Hello World from CUDA!


In [15]:
!ls /root/.cache/torch_extensions/py311_cu124/inline_ext

build.ninja  cuda.cu  cuda.cuda.o  inline_ext.so  main.cpp  main.o
